# Bradan Fleming

## Research question/interests

**How has the early response to COVID-19 affected flights at commercial airlines? How has this affected passengers?**

I want to solve this by pulling a list of consumer flights and looking for a pattern in their delays and cancellations.

If possible, I'd like to pull in a second source of data: either flights from December 2019, records of complaints about flight delays, or the massive flight data CSV from OpenSkyNetwork. Anything not related to commercial airlines or passengers is out of scope as far as I'm concerned.

I also want to read news articles to help contextualize my analysis with peoples' experiences if it's suitable.

### Bar charts I want to make (x — y)

- Date (UTC) — # of delays
    - This would be the number of delayed flights per day.
    - A day would be defined as 00:00-24:00 UTC.
- Date (UTC) — # of cancellations
    - This would be the number of cancelled flights per day.
- Date (UTC) — Total minutes of delay
    - This would be the difference between the actual and on-time schedules of the planes.
    - I might treat each flight as a sample and put this on a big scatter plot too. That could be fun.
- Date (UTC) — Estimated time wasted
    - The formula would be $(\text{total delay} + \text{# of cancellations} \cdot 30\text{ m}) \cdot \text{average # of passengers per flight}$ or something
- Date (UTC) — # of posts about "flight delays" on Twitter
    - I don't think I can do this as part of the project, but it would be fun to see.

I'd also love to make an hotmap of which US states had the most delays at a certain time because it would look cool. :^)

## Load modules

In [ ]:
import pandas as pd

## Load raw data

In [ ]:
df = pd.read_csv('../data/raw/Jan_2020_ontime.csv')
df.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,DEST,DEP_TIME,DEP_DEL15,DEP_TIME_BLK,ARR_TIME,ARR_DEL15,CANCELLED,DIVERTED,DISTANCE,Unnamed: 21
0,1,3,EV,20366,EV,N48901,4397,13930,1393007,ORD,...,GRB,1003.0,0.0,1000-1059,1117.0,0.0,0.0,0.0,174.0,NaN
1,1,3,EV,20366,EV,N16976,4401,15370,1537002,TUL,...,ORD,1027.0,0.0,1000-1059,1216.0,0.0,0.0,0.0,585.0,NaN
2,1,3,EV,20366,EV,N12167,4404,11618,1161802,EWR,...,TYS,1848.0,0.0,1800-1859,2120.0,0.0,0.0,0.0,631.0,NaN
3,1,3,EV,20366,EV,N14902,4405,10781,1078105,BTR,...,IAH,1846.0,0.0,1800-1859,2004.0,0.0,0.0,0.0,253.0,NaN
4,1,3,EV,20366,EV,N606UX,4407,14524,1452401,RIC,...,IAH,1038.0,0.0,1000-1059,1330.0,0.0,0.0,0.0,1157.0,NaN
